In [3]:
# a =  0.4420450177378683
# b = 0.010690629668597906
# c = 0.060119161083589945

a =  0.009642943936851278
b = 0.0002571122530929948
c = 0.4094753693994586

x = a + b + c + 1.0
w1 = a / x
w2 = (a + b) / x
w3 = (a + b + c) / x
w4 = 1.0
weights = [w1, w2, w3, w4]
print(weights)

[0.0067937937792934494, 0.006974938421125069, 0.29546476431015783, 1.0]


In [12]:
# assume you already have step_2_losses from your find_min_s2_loss logic
# e.g.:
import re

log_path = '/home/cherish/SADD/sfd-main/exps/00054-cifar10-4-3-dpmpp-3-poly7.0/log.txt'
all_losses = [[] for _ in range(3)]
pattern = re.compile(r"Step:\s*(\d+)\s*\|\s*Loss-mean:\s*([0-9]+\.[0-9]+)")
with open(log_path, 'r') as f:
    for line in f:
        m = pattern.search(line)
        if m:
            step = int(m.group(1))
            loss = float(m.group(2))
            if 0 <= step < 3:
                all_losses[step].append(loss)

step_2_losses = all_losses[2]

# now find the min and its iteration index
min_loss = min(step_2_losses)
min_iter = step_2_losses.index(min_loss)
print(f"Min Step-2 loss {min_loss:.6f} occurred at iteration index {min_iter}")

Min Step-2 loss 4.244495 occurred at iteration index 1133


In [10]:
find_min_s2_loss("/home/cherish/SADD/sfd-main/exps/00054-cifar10-4-3-dpmpp-3-poly7.0/log.txt")

1563


4.24449539